In [37]:
from bs4 import BeautifulSoup
import requests
import sqlite3
from sqlite3 import Error

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import re

In [38]:
chrome_driver_path = 'C:\Developer\chromedriver'
driver = webdriver.Chrome(executable_path=chrome_driver_path)

C:\Users\Charlie\AppData\Local\Temp\ipykernel_23468\2495893305.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


In [39]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn

In [40]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [41]:
conn = create_connection("parkruns.db")

sql_create_parkrun_table = """ CREATE TABLE IF NOT EXISTS parkruns (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL UNIQUE,
                                    postcode text
                                ); """

if conn is not None:
    # create projects table
    create_table(conn, sql_create_parkrun_table)

else:
    print("Error! cannot create the database connection.")

In [42]:
def create_parkrun(conn, parkrun, parkrun_postcode):
    """
    Create a new project into the parkruns table
    :param conn:
    :param parkrun:
    :return: project id
    """
    sql = ''' INSERT INTO parkruns(name, postcode)
              VALUES(?,?) '''
    cur = conn.cursor()
    
    # When you are inserting one column value into each row, insert value like this so is interpreted as a tuple and not as an input sequence
    cur.execute(sql, (parkrun, parkrun_postcode))
    conn.commit()
    return cur.lastrowid

In [43]:
def select_all_parkruns(conn):
    """
    Query all rows in the parkruns table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM parkruns")

    rows = cur.fetchall()

    for row in rows:
        print(row)

In [44]:
def get_postcode(driver, parkrun_name):
    park_name = parkrun_name[:-3]

    search_text = f'{park_name} post code'

    driver.get('https://www.google.com/')

    driver.find_element_by_xpath('//*[@id="L2AGLb"]').click()

    google_search_input_field = driver.find_element(by=By.NAME, value='q')
    google_search_input_field.send_keys(search_text)
    google_search_input_field.send_keys(Keys.RETURN)

    # Fun fact: the href attribute of the 28th (27th index) anchor tag on a google search webpage is the link to the first search result:
    
    urls = [anchor.get_attribute('href') for anchor in driver.find_elements(by=By.CSS_SELECTOR, value='div > a')]
    # print([str(url) for url in urls])


    output = ''
    blacklist = [
    '[document]',
    'noscript',
    'html',
    'meta',
     'head', 
    'input',
    'script',
    ]

    for url in urls:
        if url != None and re.search(r'google', str(url)) == None:
            res = requests.get(url)
            html_page = res.content
            soup = BeautifulSoup(html_page, 'html.parser')
            text = soup.find_all(text=True)
            for t in text:
                output += '{} '.format(t)
            if output != '':
                break

    postcode_format_regex_list = [
        r'[A-Z]{2}[0-9]{1}[A-Z]{1}\s[0-9]{1}[A-Z]{2}', 
        r'[A-Z]{1}[0-9]{1}[A-Z]{1}\s[0-9]{1}[A-Z]{2}', 
        r'[A-Z]{1}[0-9]{1}\s[0-9]{1}[A-Z]{2}',
        r'[A-Z]{1}[0-9]{2}\s[0-9]{1}[A-Z]{2}', 
        r'[A-Z]{2}[0-9]{1}\s[0-9]{1}[A-Z]{2}', 
        r'[A-Z]{2}[0-9]{2}\s[0-9]{1}[A-Z]{2}']

    print(output)

    for postcode_format in postcode_format_regex_list:
        if re.search(postcode_format, output):
            parkrun_postcode = re.search('[A-Z]{2}[0-9]{2}\s[0-9]{1}[A-Z]{2}', output)[0]
            print(f'Park run: {parkrun_name}, Postcode: {parkrun_postcode}')

        return parkrun_postcode

In [45]:
url = "https://en.wikipedia.org/wiki/List_of_Parkruns_in_the_United_Kingdom"
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

In [46]:
# rows = soup.find_all("tr")[1:]

# for i in range(len(rows)):
#     try:
#         parkrun_name = rows[i].find('td').text

#         if '[' in parkrun_name:
#             parkrun_name = parkrun_name.split('[')[0]
#         if '\t' in parkrun_name:
#             parkrun_name = parkrun_name.replace('\t', ' ')
#         if '\n' in parkrun_name:
#             parkrun_name = parkrun_name.replace('\n', '')
#         if '\xa0' in parkrun_name:
#             parkrun_name = parkrun_name.replace('\xa0', ' ')

#         # TODO: Refactor the above in regex

#         parkrun_post_code = get_postcode(driver, parkrun_name)
        
#         create_parkrun(conn, parkrun_name, parkrun_post_code)

#     except Exception as e:
#         print(e)

get_postcode(driver, 'Bromley parkrun')

C:\Users\Charlie\AppData\Local\Temp\ipykernel_23468\4087058162.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="L2AGLb"]').click()


html 
 
 
 Access denied 
 
 
 
 
 
 
(function(){if(document.addEventListener&&window.XMLHttpRequest&&JSON&&JSON.stringify){var e=function(a){var c=document.getElementById("error-feedback-survey"),d=document.getElementById("error-feedback-success"),b=new XMLHttpRequest;a={event:"feedback clicked",properties:{errorCode:1020,helpful:a,version:3}};b.open("POST","https://sparrow.cloudflare.com/api/v1/event");b.setRequestHeader("Content-Type","application/json");b.setRequestHeader("Sparrow-Source-Key","c771f0e4b54944bebf4261d44bd79a1e");
b.send(JSON.stringify(a));c.classList.add("feedback-hidden");d.classList.remove("feedback-hidden")};document.addEventListener("DOMContentLoaded",function(){var a=document.getElementById("error-feedback"),c=document.getElementById("feedback-button-yes"),d=document.getElementById("feedback-button-no");"classList"in a&&(a.classList.remove("feedback-hidden"),c.addEventListener("click",function(){e(!0)}),d.addEventListener("click",function(){e(!1)}))})}})();
 


UnboundLocalError: local variable 'parkrun_postcode' referenced before assignment

In [ ]:
def do_stuff():

    # res = requests.get('https://charlie-lindsay.netlify.app/')
    res = requests.get('https://www.streetcheck.co.uk/postcode/br13sf#:~:text=Bromley%20Park%20in%20Bromley%20is,constituency%20of%20Bromley%20and%20Chislehurst.')
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    text = soup.find_all(text=True)

    output = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
    ]

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)

    print(output)

In [ ]:
do_stuff()